# Mimetic regridding from lon-lat to cubed-sphere
This notebook shows how to regrid/remap edge or face centred data on a lon-lat grid to cubed-sphere in a flux conservative way.

In [1]:
import numpy
import mint
print(f'mint version = {mint.__version__}')

mint version = 1.19.1


## Create a regridding instance

In [2]:
regridder = mint.RegridEdges()

## Set the flags for the source and destination grids
For the lon-lat grid we recommend $fixLonAcrossDateline=0$ and $averageLonAtPole=0$. For the cibed-sphere grid we recommend $fixLonAcrossDateline=1$ and $averageLonAtPole=1$. The first option will add and subtract 360 degrees, 
to reduce the length of cell edges when this cross the dateline. The second option will adjust the longitudes at the poles to make the grid compact.

In [3]:
regridder.setSrcGridFlags(fixLonAcrossDateline=0, averageLonAtPole=0)  # for lon-lat
regridder.setDstGridFlags(fixLonAcrossDateline=1, averageLonAtPole=1)  # for cubed-sphere

## Load the source and destination grids
The grid are assumed to be stored in the Ugrid 2D netCDF format. Note that we need to specify the name of the grid in the file as well, as several grids ccan live inside the file. This is achieved by passing the name of the file folowwed by '$' and the name of the grid (or mesh).

In [4]:
regridder.loadSrcGrid('../../data/latlon100x50.nc$latlon')  # mesh name is latlon
regridder.loadDstGrid('../../data/lfric_diag_wind.nc$Mesh2d')  # mesh name is Mesh2d

## Compute the regridding weights
Next we compute the weights by building the regridder. Option $debug=1$ will print all the destination edges that partially fall outside the source domain. Option $periodX$ is useful for global domains. Option $numCellsPerBucket$ sets the number of cell in each cell locator bucket. The latter affects performance only, except in the edge cases. We recommend anywhere 10-500 cells per bucket.

In [5]:
regridder.build(numCellsPerBucket=128, periodX=360., debug=1)

## View the log messages
It's good practice to view the log messages to chak that all destination edges were found inside the source grid.

In [6]:
mint.writeLogMessages('log.txt')
!  cat log.txt

[Thu Nov 11 16:25:19 2021 info    ]  src/mntRegridEdges.cpp in function mnt_regridedges_loadSrcGrid (line 468): loading src grid from file "../../data/latlon100x50.nc$latlon"
[Thu Nov 11 16:25:19 2021 info    ]  src/mntRegridEdges.cpp in function mnt_regridedges_loadDstGrid (line 481): loading dst grid from file "../../data/lfric_diag_wind.nc$Mesh2d"


## Saving the weights to file

In [7]:
regridder.dumpWeights('lonlat_cs_weights.nc')